### Calculate Navigation Differences Between Acclimation and Experiment Periods Per Larvae
#### List of tasks accomplished in this Jupyter Notebook:
- Calculate the change in each of the ten navigation variables for each animal between the acclimation period and the experiment period. Save the results as a separate data file. 
	1. Time spent moving
	2. Total distance traveled
	3. Mean speed when moving
	4. Maximum speed
	5. Mean speed during the first minute 
	6. Difference in mean speed (first and last minute)
	7. Number of sharp turns (>45 degrees)
	8. Number of spiral movements
	9. Number of continuous paths that are not spirals
	10. Longest rest period (in seconds)

In [1]:
import numpy as np
import pandas as pd
import math, os
import more_itertools as mit

In [2]:
fps = 2
sh = 1 # direction in which to shift vector for delta calculations
threshold = 2 # consecutive seconds spent moving
still_threshold = 10 # seconds for minimum still num count
angle_thresh = 20 # Allowed error from previous angle
sharp_angle = 45
spiral_thresh = 4
spiral_speed_thresh = np.inf

In [3]:
df = pd.read_csv("./data/experiment_IDs/cleaned_static_data.csv")
animals = df["animal_ID"].unique()
# Check that animal IDs in dataframe are unique
assert len(df) == len(animals)

fps = 2
master_df = pd.DataFrame()

for index, row in df.iterrows():
    animal = row['animal_ID']
    aID = animal[:9]
    pos = animal[10:]
    
    tags = ['A', 'E']
    taglists = []
    
    filename1 = "./data/trajectories/video_calculations/"+aID+'-A-'+pos+'.csv'
    filename2 = "./data/trajectories/video_calculations/"+aID+'-E-'+pos+'.csv'
    if (os.path.isfile(filename1)) and (os.path.isfile(filename2)):
        for tag in tags:
            filename = "./data/trajectories/video_calculations/"+aID+'-'+tag+'-'+pos+'.csv'
            temp = pd.read_csv(filename)

            # Get the change in the change in angle 
            temp['angle_delta_2'] = temp['angle_delta'].shift(sh)
            temp['angle_delta_delta'] = temp['angle_delta']-temp['angle_delta_2']
            temp['angle_delta_delta'] = np.abs(temp['angle_delta_delta'])
            temp_move = temp[temp["moving"] == True]

            # % TIME SPENT MOVING, percent ------------------------------
            if len(temp_move) > 0:
                time_move = 100*len(temp_move)/len(temp)
            else:
                time_move = 0

            # TOTAL DISTANCE TRAVELED, in meters
            total_dist = temp['speed_mm_s'].sum()/fps/1000

            # MAXIMUM SPEED
            max_speed = temp['speed_BL'].max()

            # AVERAGE SPEED FOR FIRST MINUTE
            mean_speed_first = temp[temp['frames'] <= 60*fps]['speed_BL'].mean()

            # DIFFERENCE BETWEEN FIRST and LAST MINUTES
            diff_speed_f_l = temp[temp['frames'] >= 14*60*fps]['speed_BL'].mean() - mean_speed_first

            # AVERAGE SPEED WHEN MOVING
            if len(temp_move) == 0:
                avg_speed = 0
            else:
                avg_speed = temp_move['speed_BL'].mean()

            # NUMBER OF SLOW MOVING SPIRALS
            temp_move_side1 = temp_move[temp_move['angle_delta'] < 0]
            temp_move_side2 = temp_move[temp_move['angle_delta'] > 0]
            timestamps1 = [list(x) for x in mit.consecutive_groups(temp_move_side1.index)]
            timestamps2 = [list(x) for x in mit.consecutive_groups(temp_move_side2.index)]
            timestamps1 = [x for x in timestamps1 if len(x) > spiral_thresh*fps]
            timestamps2 = [x for x in timestamps2 if len(x) > spiral_thresh*fps]
            small_spirals, small_spiral_len, total_timestamps = 0, [], []
            for thresh_time in timestamps1+timestamps2: 
                thresh_time = [thresh_time[0]-1]+thresh_time
                df_thresh = temp[temp.index.isin(thresh_time)]
                if np.mean(df_thresh['speed_mm_s']) < spiral_speed_thresh:
                    small_spirals += 1
                    small_spiral_len.append(df_thresh['speed_mm_s'].sum())
                    total_timestamps += df_thresh['frames'].tolist()

            # NUMBER OF SHARP TURNS
            temp_sharp = temp[temp['speed_mm_s'] > 4]
            temp_sharp = temp_sharp[(temp_sharp['angle_delta'] < -sharp_angle) | \
                                    (temp_sharp['angle_delta'] > sharp_angle)]
            temp_sharp = [x-1 for x in temp_sharp['frames'].tolist()]
            temp_sharp = temp[temp['frames'].isin(temp_sharp)]
            sharp_turns = len(temp_sharp)*100/len(temp)

            # NUMBER OF CONTINUOUS PATHS
            # PROPORTION OF DISTANCE MOVING CONTINUOUSLY, INCLUDING SPIRALS
            temp_move_cons = temp_move[temp_move['angle_delta_delta'] <= angle_thresh]
            timestamps = [list(x) for x in mit.consecutive_groups(temp_move_cons.index)]
            continuous_nums = 0
            for thresh_time in timestamps: 
                thresh_time = [thresh_time[0]-1]+thresh_time
                df_thresh = temp[temp.index.isin(thresh_time)]
                # Remove spirals using total_timestamps 
                df_thresh = df_thresh[~df_thresh['frames'].isin(total_timestamps)]
                continuous_nums += 1

            # MAXIMUM REST LENGTH IN SECONDS
            temp_still = temp[temp["moving"] == False]
            timestamps = [list(x) for x in mit.consecutive_groups(temp_still.index)]
            thresh_times = [x for x in timestamps if len(x) > still_threshold*fps]
            still_len = []
            for thresh_time in thresh_times:
                thresh_time = [thresh_time[0]-1]+thresh_time
                df_thresh = temp[temp.index.isin(thresh_time)]
                still_len.append(len(df_thresh))
            if len(still_len) > 0:
                still_len_max = np.max(still_len)/fps # in seconds
            else:
                still_len_max = 0

            taglists.append([time_move, total_dist, avg_speed, max_speed,
                             mean_speed_first, diff_speed_f_l, 
                             sharp_turns, small_spirals, continuous_nums, 
                             still_len_max])

        temp = pd.DataFrame({
            # STATIC VARIABLES ----------------------------
            "animal_ID": [row["animal_ID"]], 
            "treatment_odor": row["treatment_odor"],
            "sex": row["sex"], 
            "species": row['species'],
            'dead':row['dead'],
            "length_mm": row['larvae_length_mm'],

            # VARIABLES CALCULATED THROUGHOUT VIDEO -------
            "D_time_move_p": taglists[1][0] - taglists[0][0],
            "D_total_dist_m": taglists[1][1] - taglists[0][1],
            "D_avg_speed_BL": taglists[1][2] - taglists[0][2],
            "D_max_speed_BL": taglists[1][3] - taglists[0][3],
            "D_mean_speed_first_BL": taglists[1][4] - taglists[0][4],
            "D_diff_speed_first_last_BL": taglists[1][5] - taglists[0][5],
            "D_sharp_turns_p": taglists[1][6] - taglists[0][6],
            "D_spirals": taglists[1][7] - taglists[0][7],
            "D_continuous": taglists[1][8] - taglists[0][8],
            "D_max_still_sec": taglists[1][9] - taglists[0][9],
            })
        master_df = pd.concat([master_df, temp])

master_df.to_csv('./data/trajectories/cleaned_animal_analyses_diffs.csv', index=False)
display(master_df.head())
print('--- Finished with saving df! ---') 

,animal_ID,treatment_odor,sex,species,dead,length_mm,D_time_move_p,D_total_dist_m,D_avg_speed_BL,D_max_speed_BL,D_mean_speed_first_BL,D_diff_speed_first_last_BL,D_sharp_turns_p,D_spirals,D_continuous,D_max_still_sec
0,190626-01-top,100ul_milliQ_water,f,Aedes aegypti,no,5.2,-4.600199,-0.504155,-0.056901,-1.322557,-0.151460,-0.200682,-2.237329,2,-3,0.0
0,190626-01-bottom,100ul_milliQ_water,f,Aedes aegypti,no,5.9,2.777778,0.826280,0.148453,0.429486,0.696910,-1.063525,-1.333333,3,16,-16.0
0,190626-02-top,100ul_milliQ_water,m,Aedes aegypti,no,5.2,0.111235,-0.023373,0.006451,0.125519,-0.004963,-0.017658,0.000000,0,2,27.5
0,190626-02-bottom,100ul_milliQ_water,m,Aedes aegypti,no,5.3,18.965517,0.677625,0.098265,-0.096798,-0.229486,0.568200,1.890990,-1,44,-116.5
0,190626-03-top,100ul_milliQ_water,missing,Aedes aegypti,missing,5.4,-22.413793,-0.990960,-0.286653,-0.606318,-0.320058,0.251430,-2.669633,0,-66,181.0


--- Finished with saving df! ---
